## W205 Project 2: Tracking User Activity

As part of an ed tech firm, I have created an assessments delivery service that would allow external data scientists from different companies to publish their assessments through the service and run queries against the data. In order to prepare the infrastructure that would allow data scientists to land the data in the necessary format and structure for querying, I have published and consumed messages through Kafka as well as transforming and landing the messages into HDFS through Spark. The report provides step-by-step instructions on how the data pipeline was spun up by using the docker-compose.yml file along with the spark SQL queries that was used for answering some basic business questions.

In order to show external data scientists the kinds of data they will have access to, I have decided to answer the following basic business questions to set the stage for any customer who would interested in publishing their assessments through our service:


<li>How many assessments are in the dataset?</li>
<li>What is the average score of the assessments in the dataset?</li>
<li>What is the standard deviation of the assessments in the dataset?</li>
<li>What is the name of our Kafka topic and how did we come up with the name?</li>

## The following commands would be executed on the Linux Command Line:

### Here are the commands that are executed once:

#### To copy in our yml file:
cp ~/w205/course-content/08-Querying-Data/docker-compose.yml .

#### To modify my yml file to allow Jupyter Notebooks for pyspark:

Updated the docker-compose.yml by adding the following entries for the expose section and ports section -
    
    expose:
      - "8888"
    ports:
      - "8888:8888"

#### To download our assessments file:
curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp

### Here are the commands that need to be executed everytime I start a cluster:

#### To check for stray containers and stray networks:
docker ps -a
docker network ls

#### To bring up the cluster:
docker-compose up -d

#### To verify the cluster:
docker-compose ps

#### To create a symbolic link in the spark container:
docker-compose exec spark ln -s /w205 w205

#### To run a Jupyter Notebook with PySpark kernel:
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark

#### To open up a Jupyter Notebook in the browser with PySpark kernel:
http://0.0.0.0:8888/?token=ba55d72339e08fead0fa77f737644674f92786b52a2f8703
http://35.197.10.250:8888/?token=ba55d72339e08fead0fa77f737644674f92786b52a2f8703

#### To create topic on kafka:
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

#### To put the assessments json on the kafka topic:
docker-compose exec mids bash -c "cat /w205/project-2-azj0718/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"

#### To check that the assessments json data is on the kafka topic:
docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"

#### To check Hadoop HDFS writes:
docker-compose exec cloudera hadoop fs -ls /tmp/
docker-compose exec cloudera hadoop fs -ls /tmp/assessments
docker-compose exec cloudera hadoop fs -ls /tmp/extracted_assessments
docker-compose exec cloudera hadoop fs -ls /tmp/my_sequences
docker-compose exec cloudera hadoop fs -ls /tmp/my_questions
docker-compose exec cloudera hadoop fs -ls /tmp/my_correct_total

#### To tear down the cluster:
docker-compose down

### Imports:

In [1]:
import json
from pyspark.sql import Row

### Check for Spark objects:

In [2]:
spark

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### Subscriber to the kafka topic 'assessments' and create a Spark dataframe:

In [4]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

### Data Cleansing by cacheing cuts on the number of warning messages:

In [5]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [6]:
raw_assessments

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

### Review the created DAG for raw assessments:

In [7]:
raw_assessments.show()

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessments|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     7|1969-12-31 23:59:...|   

### Create dataframe for assessments:

In [8]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

In [9]:
assessments.show()

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



### Register the assessments dataframe as a temporary table for executing SQL against the assessments data in memory:

In [10]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

In [11]:
extracted_assessments.registerTempTable('assessments')

In [36]:
extracted_assessments.show()

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

In [12]:
spark.sql("select keen_id from assessments limit 10").show()

+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+



In [13]:
spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()

+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                                  false|
|1516746279.3801291|                                                  false|
| 1516743820.305464|                                                  false|
|  1516743098.56811|                                                  false|
| 1516743764.813107|                                                  false|

### Data Analysis for nested multi-valued as a dictionary:

In [14]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)

In [15]:
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()

In [16]:
my_sequences.show()

+--------------------+--------------------+
|             keen_id|        sequences_id|
+--------------------+--------------------+
|5a6745820eb8ab000...|5b28a462-7a3b-42e...|
|5a674541ab6b0a000...|5b28a462-7a3b-42e...|
|5a67999d3ed3e3000...|b370a3aa-bf9e-4c1...|
|5a6799694fc7c7000...|b370a3aa-bf9e-4c1...|
|5a6791e824fccd000...|04a192c1-4f5c-4ac...|
|5a67a0b6852c2a000...|e7110aed-0d08-4cb...|
|5a67b627cc80e6000...|5251db24-2a6e-424...|
|5a67ac8cb0a5f4000...|066b5326-e547-4da...|
|5a67a9ba060087000...|8ac691f8-8c1a-403...|
|5a67ac54411aed000...|066b5326-e547-4da...|
|5a67ad9b2ff312000...|083844c5-772f-48d...|
|5a67b610baff90000...|5251db24-2a6e-424...|
|5a67ac9837b82b000...|b68128a9-6b50-41f...|
|5a67aaa4f21cc2000...|67457eec-4cad-416...|
|5a67ac46f7bce8000...|066b5326-e547-4da...|
|5a67aedaf34e85000...|7b754bca-91a1-4aa...|
|5a67aefef5e149000...|7b754bca-91a1-4aa...|
|5a67ae3f0c5f48000...|42a1e4c5-7a08-469...|
|5a67ad579d5057000...|d51a016b-0122-452...|
|5a67aae6753fd6000...|67457eec-4

In [17]:
my_sequences.registerTempTable('sequences')

In [18]:
spark.sql("select sequences_id from sequences limit 10").show()

+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+



In [19]:
spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
|5a29dcac74b662000...|1512692908.8423469|e7110aed-0d08-4cb...|
|5a2fdab0eabeda000...|1513085616.2275269|cd800e92-afc3-447...|
|5a30105020e9d4000...|1513099344.8624721|8ac691f8-8c1a-403...|
|5a3a6fc3f0a100000...| 1513779139.354213|e7110aed-0d08-4cb...|
|5a4e17fe08a892000...|1515067390.1336551|9abd5b51-6bd8-11e...|
|5a4f3c69cc6444000...| 1515142249.858722|083844c5-772f-48d...|
|5a51b21bd0480b000...| 1515303451.773272|e7110aed-0d08-4cb...|
|5a575a85329e1a000...| 1515674245.348099|25ca21fe-4dbb-446...|
+--------------------+------------------+--------------------+



### Data Analysis for nested multi-valued as a list:

In [20]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

In [21]:
my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

In [22]:
my_questions.registerTempTable('questions')

In [23]:
spark.sql("select id, my_count from questions limit 10").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+



In [24]:
spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a26ee9cbf5ce1000...|1512500892.4166169|0603e6f4-c3f9-4c2...|
|5a26ee9cbf5ce1000...|1512500892.4166169|26a06b88-2758-45b...|
|5a26ee9cbf5ce1000...|1512500892.4166169|25b6effe-79b0-4c4...|
|5a26ee9cbf5ce1000...|1512500892.4166169|6de03a9b-2a78-46b...|
|5a26ee9cbf5ce1000...|1512500892.4166169|aaf39991-fa83-470...|
|5a26ee9cbf5ce1000...|1512500892.4166169|aab2e817-73dc-4ff...|
+--------------------+------------------+--------------------+



### Data Analysis for handling holes in the JSON data by removing level of indirection through flat map:

In [25]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [26]:
my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()

In [27]:
my_correct_total.registerTempTable('ct')

In [28]:
spark.sql("select * from ct limit 10").show()

+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
+-------+-----+



In [29]:
spark.sql("select correct / total as score from ct limit 10").show()

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+



In [31]:
spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



## Business Questions:

### Assumption: keen_id is unique

#### Question 1: How many assessments are in the correct total dataset?
#### Answer: As you can see, there was a total of 3,275 assessments in the correct total dataset.

In [35]:
spark.sql("select count(*) from ct").show()

+--------+
|count(1)|
+--------+
|    3275|
+--------+



#### Question 2: What is the average score of the assessments in the correct total dataset?
#### Answer: As you can see, the average score happened to be approximately 62.66.

In [32]:
spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()

+-----------------+
|        avg_score|
+-----------------+
|62.65699745547047|
+-----------------+



#### Question 3: What is the standard deviation of the correct total dataset?
#### Answer: As you can see, the standard deviation was approximately 0.31.

In [48]:
spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



#### Question 4: What is the name of my kafka queue?
#### Answer: The name of my kafka queue is assessments because the main reason for creating the kafka topic was to publish "assessments" json data to the kafka topic using kafkacat. 

### For each dataframe, we will write out to Hadoop HDFS in parquet format to allow us to build a batch and serving layer for external queries:

In [34]:
assessments.write.mode('overwrite').parquet("/tmp/assessments")
extracted_assessments.write.mode('overwrite').parquet("/tmp/extracted_assessments")
my_sequences.write.mode('overwrite').parquet("/tmp/my_sequences")
my_questions.write.mode('overwrite').parquet("/tmp/my_questions")
my_correct_total.write.mode('overwrite').parquet("/tmp/my_correct_total")

### To check that we wrote the files to HDFS in the linux Command Line:

docker-compose exec cloudera hadoop fs -ls /tmp/

docker-compose exec cloudera hadoop fs -ls /tmp/assessments

docker-compose exec cloudera hadoop fs -ls /tmp/extracted_assessments

docker-compose exec cloudera hadoop fs -ls /tmp/my_sequences

docker-compose exec cloudera hadoop fs -ls /tmp/my_questions

docker-compose exec cloudera hadoop fs -ls /tmp/my_correct_total

### To tear down the cluster:

docker-compose down